In [11]:
#!pip install duckdb

In [5]:
import requests # -> Để call API
import json # -> Xử lý file JSON
import pandas as pd # -> Thư viện xử lý dữ liệu dạng bảng
import re # -> Thư viện xử lý text: regular expressions
from datetime import datetime as dt # -> Thư viện xử lý dữ liệu thời gian
import duckdb # -> Thư viện "giả lập" xử lý dữ liệu bằng SQL 

### STEP 0: XEM LẠI DATA ĐÃ CÓ (SLACK API)
Ở bước đầu tiên, ta xem lại tất cả các data của DATACracy Atom mà ra đã biết:

Data từ Slack API:

- Danh sách thành viên
- Danh sách các channels

Lịch sử tin nhắn trên các channels

- Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet):
- Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)

===> Các dữ liệu này lần lượt được lấy bằng code bên dưới.

In [6]:
## Load Token file 
with open('../env_variable.json', 'r') as j:
    json_data = json.load(j)

### 0.1. Pull List of Members

In [11]:
import numpy as np

In [12]:
# 1. LIST OF SLACK MEMBERS 

## Pull list of member as JSON
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(json_data['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

## Convert to CSV
user_dict = {'user_id':[],'name':[],'display_name':[],'real_name':[],'title':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

user_df = pd.DataFrame(user_dict) 
user_df = user_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
user_df.head()

,user_id,name,display_name,real_name,title,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False


### 0.2. List of Channels

In [13]:
# 2. LIST OF SLACK CHANNELS

endpoint2 = "https://slack.com/api/conversations.list"
data = {'types': 'public_channel,private_channel'} # -> CHECK: API Docs https://api.slack.com/methods/conversations.list/test
response_json = requests.post(endpoint2, headers=headers, data=data).json() 
channel_dat = response_json['channels']

channel_dict = {'channel_id':[], 'channel_name':[], 'is_channel':[],'creator':[],'created_at':[],'topics':[],'purpose':[],'num_members':[]}
for i in range(len(channel_dat)):
  channel_dict['channel_id'].append(channel_dat[i]['id'])
  channel_dict['channel_name'].append(channel_dat[i]['name'])
  channel_dict['is_channel'].append(channel_dat[i]['is_channel'])
  channel_dict['creator'].append(channel_dat[i]['creator'])
  channel_dict['created_at'].append(dt.fromtimestamp(float(channel_dat[i]['created'])))
  channel_dict['topics'].append(channel_dat[i]['topic']['value'])
  channel_dict['purpose'].append(channel_dat[i]['purpose']['value'])
  channel_dict['num_members'].append(channel_dat[i]['num_members'])

channel_df = pd.DataFrame(channel_dict) 
channel_df = channel_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
channel_df.head()

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 17:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 14:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 10:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 14:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-17 00:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


### 0.3. Message Data

In [14]:
endpoint3 = "https://slack.com/api/conversations.history"

In [15]:
msg_dict = {'channel_id':[],'msg_id':[], 'msg_ts':[], 'user_id':[], 'latest_reply':[],'reply_user_count':[],'reply_users':[],'github_link':[]}
for channel_id, channel_name in zip(channel_df['channel_id'], channel_df['channel_name']):
  print('Channel ID: {} - Channel Name: {}'.format(channel_id, channel_name))
  try:
    data = {"channel": channel_id} 
    response_json = requests.post(endpoint3, data=data, headers=headers).json()
    msg_ls = response_json['messages']
    for i in range(len(msg_ls)):
      if 'client_msg_id' in msg_ls[i].keys():
        msg_dict['channel_id'].append(channel_id)
        msg_dict['msg_id'].append(msg_ls[i]['client_msg_id'])
        msg_dict['msg_ts'].append(dt.fromtimestamp(float(msg_ls[i]['ts'])))
        msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'] if 'latest_reply' in msg_ls[i].keys() else 0))) ## -> No reply: 1970-01-01
        msg_dict['user_id'].append(msg_ls[i]['user'])
        msg_dict['reply_user_count'].append(msg_ls[i]['reply_users_count'] if 'reply_users_count' in msg_ls[i].keys() else 0)
        msg_dict['reply_users'].append(msg_ls[i]['reply_users'] if 'reply_users' in msg_ls[i].keys() else 0) 
        ## -> Censor message contains tokens
        text = msg_ls[i]['text']
        github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text)
        msg_dict['github_link'].append(github_link[0] if len(github_link) > 0 else np.nan)
  except:
    print('====> '+ str(response_json))

Channel ID: C01B4PVGLVB - Channel Name: general
Channel ID: C01BYH7JHB5 - Channel Name: contents
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01CAMNCJJV - Channel Name: branding-design
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01U6P7LZ8F - Channel Name: atom-assignment1
Channel ID: C01UL6K1C7L - Channel Name: atom-week1
Channel ID: C01ULCHGN75 - Channel Name: atom-general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C020VMT58JK - Channel Name: topics-data-analytics
Channel ID: C0213MNH9L6 - Channel Name: topics-python
Channel ID: C0213N56M2A - Channel Name: topics-materials
Channel ID: C021FSDN7LJ - Channel Name: atom-assignment2
Channel ID: C021KLB0DSB - Channel Name: discuss-group3
Channel ID: C021KLB90GP - Channel Name: discuss-group4
Channel ID: C02204B2CD6 - Channel Name: atom-week2
Channel ID: C0220KU9399 - Channel Name: discuss-group1
Channel ID: C0226D3LEQ4 - Channel Name: atom-week3
Channel ID: C0227A51SAY - Channel Name: atom-ass

In [16]:
msg_df = pd.DataFrame(msg_dict)
msg_df = msg_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)
msg_df.tail(3)

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
257,C022RRWQ6US,231fad01-7083-4cd0-b6b6-e97bf54779e2,2021-05-23 06:23:13.001400,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,https://github.com/anhdanggit/
258,C022Y1FUETE,ee582f3d-c339-4d06-a263-b71c6470e647,2021-05-24 18:49:57.001000,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,0,NaN
259,C022Y1FUETE,38adf48f-21cc-437c-9ad3-31cd002bc4f8,2021-05-23 14:58:05.001500,U01BE2PR6LU,2021-05-23 23:15:45.002200,1,[U01VB632FPS],NaN



### 0.4. DataCracy Info
Data do dự án tự collect - File CSV (trích xuất từ Google Spreadsheet): Danh sách thành viên được phân theo vị trí (mentors, learners, BTC)

Trong cùng folder Github assignment_4

In [31]:
dtc_groups = pd.read_csv('datacracy_groups.csv')
dtc_groups.head()

,name,DataCracy_role
0,slackbot,BTC
1,loclexuan26392,BTC
2,locle.ds,BTC
3,maianhdang.ftu,BTC
4,huyenhoang.design,BTC


### STEP 1: NHU CẦU & MỤC ĐÍCH
Đặt mình vào vị trí người chủ, bạn quan tâm đến điều gì?

- Quan trọng nhất của mọi Data Solution, là bắt đầu từ nhu cầu, mục đích và câu hỏi lớn của Clients (người chủ).

- Chính từ những câu hỏi lớn này, ta có thể khoanh vùng thông tin nào quan trọng, ta muốn đạt được điều gì?

## TODO#1: Requirements
Tự trả lời các câu hỏi sau, từ góc nhìn của bạn (đặt mình vào vị trí bạn là co-founder của dự án DataCracy):

1. Mục đích của lớp học Atom là gì?

2. BTC sẽ quan tâm đến những chủ đề/quy trình gì để đạt được Mục Đích trong (1)?

3. Làm sao để đo lường các điểm trong (2)? => Metrics?

4. Dựa vào các data đã có như liệt kê trong STEP 0:

- Chỉ dùng những data sẵn có, ta có thể đo lường và thiết kế những metrics nào bạn đã liệt kê trong (3)?

- Tham khảo Slack API và hình dung về các thông tin DataCracy có khả năng thu thập, bạn sẽ đề xuất DataCracy thu thập thêm những thông tin gì?

In [ ]:

## Điền vào bên dưới câu trả lời của bạn
'''
1. Mục đích
a. Phổ cập các kỹ năng sử dụng data cho learner những người có mong muốn  tìm hiểu về Data hoặc phát triển công việc có liên quan đến dữ liệu. bằng hình thức flipped learning với sự hướng dẫn từ DataCracy. 
b. Chọn được 20 học viên có đủ kỹ năng, tư duy và thái độ học tốt để tham gia vào lớp học Scala. Áp dụng được kiến thức từ lớp Atom để giải quyết các bài toán thực tế của danh nghiệp SME

2. Chủ đề cần quan tâm
a. Học viên hiểu được mục tiêu khóa học, chất lượng buổi học có phù hợp với kỹ năng và mong muốn của học viên chưa?
b. Mức độ tiếp thu của người học, kỹ năng tư duy của học viên khi làm việc với data và các tool để làm việc với data.
c. Sự tương tác của lớp học (learner - learner, learner - mentor, teacher)
d. Những học viên nổi trội về kỹ năng, tư duy logic hoặc chủ động nghiên cứu, hoàn thành bài tập tốt (tiêu chí để chọn 20 học viên)

Quy trình : Thiết kế bài giảng dựa trên kinh nghiệm của teacher  –>  đánh giá mức độ tiếp thu của học viên sau mỗi lớp và bài tập + nhận ý kiến đóng góp và khó khăn của học viên –> chỉnh sửa và cập nhật lại bài giảng cho phù hợp sau mỗi lớp học

3. Đo lường các tiêu chí đánh giá của (2)
Metrics 2a
-	Số lượng học viên tham gia lớp học (từ lúc tham gia đến khi kết thúc lớp Atom)
-   Đánh giá sự hài lòng và chất lượng lớp học (khảo sát học viên)
Metrics 2b
-	Tỉ lệ nộp bài tập của học viên qua từng assignment (%)
-	Thời gian hoàn thành bài tập so với deadline (có phải thời gian càng lâu thì bài tập càng khó đối với học viên?)
-   % bài tập được thực hiện của mỗi học viên (dựa trên review của learner hoặc mentor)
-	Sự quan tâm của học viên đối với mỗi topic (số msg được discuss trogn từng topic)
Metrics 2c
-	Tổng số lượng câu hỏi discuss và reply trong mỗi tuần/group or topic
-   Tỉ lệ review bài tập của học viên (count số review của user sau từng assignment, đảm bảo rule: 1 user review 2 bài)
-   Tỉ lệ học viên tương tác trên tổng số học viên
Metrics 2d
-  Số lượng tương tác thông qua msg của mỗi học viên sau khi kết thúc Atom (nhiều nhất, trung bình, không tương tác)
-  Học viên nộp bài trễ/sớm nhất/nộp thiếu hoặc không nộp bài
-  Điểm trung bình của học viên qua sau khi kết thúc Atom

4. 
- Chỉ dùng những data sẵn có, ta có thể đo lường và thiết kế những metrics:
    Thông tin học viên tham gia
    Tỉ lệ hoàn thành bài tập và thời gian nộp bài so với deadline
    Tỉ lệ review bài tập của mỗi học viên
    Số lượng tương tác thông qua msg
    Học viên nộp bài trễ/sớm nhất/nộp thiếu hoặc không nộp bài
    Tổng số lượng câu hỏi discuss và reply trong mỗi tuần/group or topic
    Tỉ lệ học viên tương tác trên tổng số học viên
'''


### STEP 2: TỔ CHỨC THÔNG TIN

Thu thập và hệ thống lại các thông tin

In [31]:
## Hints: info() để check các thông tin (Column), số dòng (Count), và Data Type của mỗi cột
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       75 non-null     object
 1   name          75 non-null     object
 2   display_name  49 non-null     object
 3   real_name     75 non-null     object
 4   title         5 non-null      object
 5   is_bot        75 non-null     bool  
dtypes: bool(1), object(5)
memory usage: 3.1+ KB


### TODO#2: List Down

Trả lời các câu hỏi sau:

1. Có những thông tin gì trong các bảng data ở STEP 0? Ý nghĩa của mỗi trường (Column). Hints: Đọc thêm Slack API để hiểu ý nghĩa data trả về

2. Data Type của mỗi trường

3. Có NULL không? (Non-Null Count <> entries)

- Ta sẽ dùng công cụ: QuickDBD cho Assignment này => Tham khảo Sample lúc mở tool
- Copy điền đoạn text vào tools

In [ ]:
## Điền vào bên dưới câu trả lời của bạn

# Bảng user_df 
1. user.list trả về danh sách thông tin của user trong workspace. Ý nghĩa của mỗi column:
user_id	name: mỗi user có 1 ID duy nhất trong workspace
name: tên mail khi đăng ký slack
display_name: Tên user hiển thị khi tương tác
is_bot: xác định đó là account user hay là bot của user tạo
2-3. Data Type của mỗi trường
user_id string
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

# Bảng channel_df
1. conversations.list: trả về danh sách tất cả các kênh trong workspace. Ý nghĩa của mỗi column:
channel_id: mỗi channel có 1 ID duy nhất trong workspace
channel_name: tên channel trong workspace
is_channel: trả về giá trị true hoặc false để xác định đó có phải là 1 channel không
creator: người tạo channel
created_at: thời gian tạo channel
topics: chủ đề của channel
purpose: mục đích của channel
num_members: số lượng user tham gia trong channel

2-3. Data Type của mỗi trường
channel_id string
channel_name string
is_channel bool
creator string
created_at datetime64
topics NULL string
purpose NULL string
num_members int

# Bảng msg_df
1. conversations.history trả về nội dung và hoạt động của các tin nhắn từ các channel. Ý nghĩa của mỗi column:
msg_id: ID của mỗi msg
msg_ts: thời gian gửi msg
user_id: ID user gửi msg
latest_reply: thời gian reply msg cuối cùng
reply_user_count: số lượng người reply trên msg
reply_users: ID user đã reply msg
github_link: kiểm tra trong msg có link github không

2-3. Data Type của mỗi trường
channel_id string FK >- channel_df.channel_id
msg_id string
msg_ts datetime64
user_id string
latest_reply datetime64
reply_user_count int
reply_users arr
github_link NULL string

# Bảng dtc_group
name: tên mail khi đăng ký slack
DataCracy_role: vai trò của từng name

### STEP 3: NGUYÊN TẮC CẦN ÁP DỤNG
TODO#3: Rules & Logics

Dựa vào các kiến nghị TODO#1 và Quan sát ở TODO#2, bạn có những đề xuất gì về rules trong vận hành để cải thiện thông tin và quan sát?

Về mặt vận hành

    Rules gì cần áp dụng cho cách thức nhập data?

Một số gợi ý

    Rules để cải thiện tỷ lệ NULL trong các bạn? (Ví dụ: Bắt buộc nhập các thông tin trên Slack? Có cần thiết không?)

    Làm sao để xác định message nào là bài submit assigment? Message nào là review? Message vào là các nội dung không liên quan? (Ví dụ: Users để tag #submit, #review?)
...

Về mặt data
    Logics gì cần áp dụng để kiểm tra sự hợp lý của Data?

Một số gợi ý

    Hai users trùng tên?
    Ngày latest reply > ngày post?
    Ngày post trong năm 2021 (sau khi dự án DataCracy thành lập)
    Mentor Group nào sẽ chỉ post trong channel discussion của group đó?
...
Bạn có thể đưa các đề xuất để đưa vào vận hành nhằm cải thiện data và giúp bạn đo lường theo dõi các metrics đã được liệt kể trong TODO#1

In [ ]:
Rules về mặt vận hành
1.	Tạo user
-	display_name: cập nhật lại thông tin
-	real_name: nên để tên thật để dễ kiểm soát và đối chiếu với thông tin đã đăng ký với BTC
-	Title: nên cập nhật title cho mỗi user để phân loại (learner, BTC, mentor)
2.	Channel
-	Bổ sung nội dung topic và purpose sẽ giúp learner hiểu rõ chừng chức năng của mỗi channel
3.	Rules gửi msg
-	Gắn tag ở mỗi msg để dễ lọc thông tin đâu là msg gửi bài tập, discuss hay review (#submit #discuss #review)
4.	Rule gửi bài tập
-	Có nhiều bạn gửi 1 msg bài tập có nhiều link khác nhau, BTC sẽ bị miss thông tin trường hợp đó không phải là link github. Nên bạn có thể đưa các link vào phần readme để tóm tắt lại kết quả bài tập mình đã làm. Và chỉ cần gửi 1 link readme thôi.
-	Trường hợp thay đổi chỉnh sửa link thì bạn chỉ cần chỉnh trong  readme. Hoặc thay đổi link readme thì bạn edit lại msg đã gửi trước đó

Logics để quản lý data
1.	User
-	Đảm bảo ID và user là unique
-	Có thể loại bỏ user if is_bot == true
2.	Channel
-	Người tạo channel phải có trong danh sách user
-	Số lượng member trong mỗi channel <= số lượng user
-	Channel ID và Name là unique
3.	Msg
-	Latest_reply < msg_ts
-	Reply_users count == len(reply_users)
-	Mỗi user_id chỉ có 1 link github trong 1 channel assignment

### STEP 4: TỔ CHỨC BẢNG - PRIMARY KEYS

In [56]:
## Hints: nunique() để check số giá trị unique của từng trường
user_df.nunique()

user_id         75
name            75
display_name    48
real_name       73
title            5
is_bot           2
dtype: int64

### TODO#4: Tables & PK
Nhìn lại diagram của TODO#2 trong Quick DBD Diagram: Có bảng nào bạn nghĩ nên gộp lại, hay tách ra không? Vì sao?

Tìm Primary Key (PK): Unique cho từng dòng và Không NULL

Primary Key (PK) là trường giá trị/ID unique cho mỗi dòng của bảng. Hay nói cách khác, không có hai dòng trùng lặp (duplicate ID).

Fun fact để nhớ: Thử tưởng tượng nếu 2 người không quen biết, có cùng Số TK Ngân hàng :((((

In [ ]:
## Copy lại phần text của TODO#2
## Đặt PK bên cạnh col bạn chọn làm PK
Gộp thông tin bảng user_df và dtc_groups -> lấy tên bảng mới: member_df. Vì 2 bảng này có cùng thuộc tính là đều trả về thông tin của user

user_df
-
user_id string pk
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

channel_df
-
channel_id string pk
channel_name string
is_channel bool
creator string
created_at datetime64
topics NULL string
purpose NULL string
num_members int

msg_df
----
channel_id string FK >- channel_df.channel_id
msg_id string pk
msg_ts datetime64
user_id string
latest_reply datetime64
reply_user_count int
reply_users arr
github_link NULL string

### STEP 5: MỐI QUAN HỆ GIỮA CÁC BẢNG
TODO#5: FK & Mapping
Như đã giới thiệu ở phần Concept, keys quan trọng trong Relational DB vì nó thể hiện mối quan hệ giữa các bạn, thông qua key, cho phép ta nối các bảng với nhau. Bây giờ ta sẽ đi tìm FK (Foreign Key):

Foreign Key: Là các keys nằm trong một bảng để liên kết với PK trong bảng khác

Đâu là các ID trong bảng, nhưng không phải là PK (do thoả điều kiện unique)?

Các ID này dẫn đến PK nào trong các bảng còn lại?

Trong các key được nối với nhau, xác định kiểu quan hệ:
- n:1 - PK ở bảng gốc lặp lại nhiều lần (nhiều dòng) ở bản chứa FK
- 1:1 - PK ở bảng gốc chỉ nối với 1 dòng

In [ ]:
## Đặt FK bên cạnh col bạn chọn làm FK
## Và thể hiện mối quan hệ bằng: id >- bảng_gốc.id (FK là PK trong bảng gốc)

user_df
-
user_id string pk
name string
display_name NULL string
real_name string
title NULL string
is_bot bool

channel_df
-
channel_id string pk
channel_name string
is_channel bool
creator string FK >- user_df.user_id #Rel: n:1
created_at datetime64
topics NULL string
purpose NULL string
num_members int

msg_df
-
channel_id string FK >- channel_df.channel_id #Rel: n:1
msg_id string pk
msg_ts datetime64
user_id string FK >- user_df.user_id #Rel: n:1
latest_reply datetime64
reply_user_count int
reply_users arr FK >- user_df.user_id #Rel: n:1
github_link NULL string


### STEP 6: DIAGRAM & TEST

### Test Database Design by SQL
Để giúp các bạn làm quen với việc thao tác trên SQL, ta sử dụng thư việc duckdb, cho phép ta thao tác với Data trên Python bằng SQL: https://duckdb.org/2021/05/14/sql-on-pandas.html

Để hiểu các code SQL bên dưới, tham khảo SQL Basic Cheatsheet, xem các phần:

Querying Single Table
- Aliases
- Filtering
- Querying Multiple Tables: Các dạng Join
- Aggregation Functions

Dựa vào DB Diagram bạn đã vẽ, sẽ giúp bạn dễ dàng theo dõi hơn trong việc join và xử lý data

In [40]:
## Đoạn code dưới join 2 bảng user_df vằ dtc_groups bằng key name => Tạo thành bảng members_df
SQL_dim_members = '''
-- Để comment trong SQL dùng -- tương đường với ## trong Python
CREATE TABLE members_df AS
SELECT 
  t1.*,
  t2.DataCracy_role
FROM user_df AS t1
JOIN dtc_groups AS t2
ON t1.name = t2.name
'''

In [50]:

SQL_dim_members = '''
CREATE TABLE members_df AS
SELECT 
  t1.DataCracy_role,
  t2.*
FROM dtc_groups AS t1
JOIN user_df AS t2
ON t1.name = t2.name
'''

In [56]:
con = duckdb.connect(database=':memory:', read_only=False) # -> Tạo DB connection
# create a table
con.execute(SQL_dim_members) # -> Chạy đoạn lệch SQL
con.execute("SELECT * FROM members_df LIMIT 5").fetch_df() # -> In 10 dòng đầu tiên của bảng members_df ra Dataframe

,DataCracy_role,user_id,name,display_name,real_name,title,is_bot
0,BTC,USLACKBOT,slackbot,Slackbot,Slackbot,NaN,False
1,BTC,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,NaN,False
2,BTC,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,NaN,False
3,BTC,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,False
4,BTC,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,NaN,False


In [33]:
con.execute("SELECT * FROM channel_df LIMIT 5").fetch_df() # -> In 5 dòng đầu tiên của bảng channel_df ra Dataframe

,channel_id,channel_name,is_channel,creator,created_at,topics,purpose,num_members
0,C01B4PVGLVB,general,True,U01BE2PR6LU,2020-09-23 17:25:03,NaN,This is the one channel that will always inclu...,65
1,C01BYH7JHB5,contents,True,U01BE2PR6LU,2020-10-05 14:28:09,NaN,NaN,12
2,C01CAMNCJJV,branding-design,True,U01AVDY7JET,2020-10-05 10:16:30,NaN,NaN,9
3,C01U6P7LZ8F,atom-assignment1,True,U01BE2PR6LU,2021-04-17 14:00:04,Assigment#1 - Python Basics &amp; Spreadsheet ...,Where learners submit assignments and give rev...,44
4,C01UL6K1C7L,atom-week1,True,U01BE2PR6LU,2021-04-17 00:44:32,TOPIC: Data Strategy. Metrics Design. Spreadsh...,TOPIC: Data Strategy. Metrics Design. Spreadsh...,43


In [286]:
con.execute("SELECT * FROM msg_df LIMIT 5").fetch_df() # -> In 5 dòng đầu tiên của bảng msg_df ra Dataframe

,channel_id,msg_id,msg_ts,user_id,latest_reply,reply_user_count,reply_users,github_link
0,C01B4PVGLVB,48ba219d-fda8-40f2-be65-00735c058071,2021-05-25 11:59:27.000600,U01BE2PR6LU,2021-05-25 13:04:44.006500,4,NaN,NaN
1,C01B4PVGLVB,89919b55-bfe9-4c15-bf45-3021048c7c28,2021-05-24 18:49:08.061100,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,NaN,NaN
2,C01B4PVGLVB,18467584-5810-4bb9-80fe-4b92af9b28d0,2021-05-23 23:26:04.057000,U01UTGT8CVA,2021-05-24 15:36:52.058100,2,NaN,NaN
3,C01B4PVGLVB,dfd2bf1a-69a1-4ac3-aacb-e2daaae8cc04,2021-05-23 14:57:36.055300,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,NaN,NaN
4,C01B4PVGLVB,fd5d6cf7-415c-459a-818c-72835fd2479a,2021-05-23 06:28:27.053900,U01BE2PR6LU,1970-01-01 07:00:00.000000,0,NaN,NaN


In [120]:
## Đoạn code dưới: Theo từng channel, count số message (phải join với bảng channel_df để lấy tên channel)
SQL = '''
SELECT 
  t2.channel_name,
  COUNT(DISTINCT t1.msg_id) AS msg_cnt
FROM msg_df AS t1
JOIN channel_df AS t2
ON t1.channel_id = t2.channel_id
GROUP BY t2.channel_name 
ORDER BY COUNT(DISTINCT t1.msg_id) DESC
'''
con.execute(SQL).fetch_df()

,channel_name,msg_cnt
0,general,42
1,atom-assignment1,42
2,atom-assignment2,42
3,atom-assignment3,36
4,discuss-group1,32
5,discuss-group3,16
6,topics-python,13
7,discuss-group4,11
8,atom-assignment4,8
9,atom-week1,6


In [110]:
## Đoạn code dưới: Lấy top 3 mentors post nhiều message nhất trong discuss-group của các nhóm
SQL = '''
WITH msg_cnt AS( ------------- > (1) Chain SQL: tạo bảng tạm thời msg_cnt: count số msg theo user, theo channel
  SELECT 
    user_id,
    channel_id,
    COUNT(msg_id) AS msg_cnt
  FROM msg_df 
  GROUP BY 1, 2
)
SELECT
  t2.real_name,
  t3.channel_name,
  t1.msg_cnt,
  t2.DataCracy_role
FROM msg_cnt AS t1
JOIN members_df AS t2 ------------ > (2) Join msg_count với members_df để lấy tên và role
ON t1.user_id = t2.user_id
JOIN channel_df AS t3 --> (3) Join với channel_df để lấy tên channel
ON t1.channel_id = t3.channel_id
WHERE t2.DataCracy_role LIKE 'Mentor%' ------------- > (4) Filter Mentors
AND t3.channel_name LIKE 'discuss-group%' ---------- > (5) Filter channel discuss theo các group
ORDER BY t1.msg_cnt DESC ---------- > (6) Sẵp xếp theo số msg từ cao xuống thấp
LIMIT 3 ------------> (7) Lấy top 3
'''
con.execute(SQL).fetch_df()

,real_name,channel_name,msg_cnt,DataCracy_role
0,Phu NDT,discuss-group1,11,Mentor_Gr1
1,Nguyễn Quang Long,discuss-group3,9,Mentor_Gr3
2,Thuy Nguyen,discuss-group4,4,Mentor_Gr4


### TODO#6: SQL¶
Thay đổi các phần trong những đoạn code SQL trên, print kết quả để hiểu về code

Tham khảo thêm SQL Basic Cheatsheet

Và, GOOGLE! + Cùng trao đổi trên Slack

Và viết SQL để trả lời các câu hỏi sau:

1. Learners groups nào hoạt động tích cực trên Slack nhất? (tính theo message count)?

2. Learners nào nộp bài sớm nhất trong Assignment 1, 2, 3?

3. Learner nào nộp bài trễ nhất trong Assignment 1, 2, 3?

4. Learner nào chưa nộp bài Assignment 3?

5. Learner nào chưa nộp bất kỳ 1 assignment nào?

6. Tỷ lệ % Learner đã nộp assignment 1, 2, 3? (*giả sử có message trong channel atom-assignment, được tính là đã submit*)

7. Tỷ lệ % Learner đã submit bài và dc review trong assignment 1, 2, 3?

8. Learners theo Group nào có tỷ lệ % hoàn thành bài tập cao nhất?

[Optional] Bạn có thể tự đặt thêm bất kỳ câu hỏi nào bạn quan tâm

In [190]:
## 1.Learners groups nào hoạt động tích cực trên Slack nhất? (tính theo message count)?
SQL_1 = '''
SELECT 
  t2.DataCracy_role,
  COUNT(DISTINCT t1.msg_id) AS msg_cnt
FROM msg_df AS t1
JOIN members_df AS t2
ON t1.user_id = t2.user_id
WHERE DataCracy_role like 'Learner%'
GROUP BY t2.DataCracy_role 
ORDER BY COUNT(DISTINCT t1.msg_id) DESC
'''
con.execute(SQL_1).fetch_df()

,DataCracy_role,msg_cnt
0,Learner_Gr4,43
1,Learner_Gr1,41
2,Learner_Gr3,34
3,Learner_Gr2,32


In [319]:
## 2.Learners nào nộp bài sớm nhất trong Assignment 1, 2, 3?
SQL_2 = '''
CREATE TABLE asgmt_submitted AS  ---------> tạo bảng chứa các thông tin learner đã nộp bài trong các channel assignment
SELECT
    DISTINCT t2.real_name,
    t2.DataCracy_role,
    t1.reply_user_count,
    t1.msg_ts,
    t3.channel_name
FROM msg_df as t1
JOIN members_df as t2 -------------> Join msg_df với members_df để lấy tên và role
ON t1.user_id = t2.user_id
JOIN channel_df as t3 ---------------> Join msg_df với channel_df để lấy channel name
ON t1.channel_id = t3.channel_id
WHERE (t3.channel_name LIKE 'atom-assignment%')  ---------> lọc channel assignment
AND (github_link IS NOT NULL) AND (t2.DataCracy_role LIKE 'Learner%') ---------> lọc row có github link và là learner
'''
con.execute(SQL_2).fetch_df()

,Count
0,90


In [320]:
print ('Người nộp bài sớm nhất:')

SQL_2a = '''
SELECT *
FROM asgmt_submitted
WHERE msg_ts == (
    SELECT
    MIN(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment1' 
)
OR msg_ts == (
    SELECT
    MIN(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment2'
)
OR msg_ts == (
    SELECT
    MIN(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment3' 
)
'''
con.execute(SQL_2a).fetch_df()

Người nộp bài sớm nhất:


,real_name,DataCracy_role,reply_user_count,msg_ts,channel_name
0,Hưng Dương Phan Nam,Learner_Gr1,2,2021-04-21 13:00:34.000800,atom-assignment1
1,Toan Tran,Learner_Gr2,1,2021-05-12 22:50:48.004000,atom-assignment2
2,Hanh Nguyen,Learner_Gr4,2,2021-05-16 19:08:08.018900,atom-assignment3


In [321]:
## 3. Nguoi nop bai muon nhat cac assignment 1 2 3
print ('Người nộp bài muộn nhất:')

SQL_3 = '''
SELECT *
FROM asgmt_submitted
WHERE msg_ts == (
    SELECT
    MAX(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment1' 
)
OR msg_ts == (
    SELECT
    MAX(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment2'
)
OR msg_ts == (
    SELECT
    MAX(msg_ts)
    FROM asgmt_submitted
    WHERE channel_name == 'atom-assignment3' 
)
'''
con.execute(SQL_3).fetch_df()

Người nộp bài muộn nhất:


,real_name,DataCracy_role,reply_user_count,msg_ts,channel_name
0,Danh Phan,Learner_Gr3,2,2021-05-10 11:51:46.004700,atom-assignment1
1,Hanh Nguyen,Learner_Gr4,1,2021-05-22 12:39:34.015500,atom-assignment2
2,Vy Le,Learner_Gr4,0,2021-05-26 16:48:44.002300,atom-assignment3


In [322]:
### 4. Learner nào chưa nộp Assignment 3?
print ('Learner chưa nộp Assignment 3:')
SQL_4 ='''
SELECT *
FROM members_df AS t1
WHERE t1.real_name NOT IN (
    SELECT 
    t2.real_name
    FROM asgmt_submitted AS t2
    WHERE channel_name == 'atom-assignment3'
    )
AND DataCracy_role LIKE 'Learner_Gr%'
'''
con.execute(SQL_4).fetch_df()

Learner chưa nộp Assignment 3:


,DataCracy_role,user_id,name,display_name,real_name,title,is_bot
0,Learner_Gr4,U01U6JM6LEB,hoangquocviet.208,Viet Hoang,Viet Hoang,NaN,False
1,Learner_Gr1,U01U6JQB695,thgiang.phan,NaN,Giang Phan,NaN,False
2,Learner_Gr1,U01U8CAMGB1,huyenngoc2017,Ngoc Huyen,Ngoc Huyen,NaN,False
3,Learner_Gr3,U01UEJG8KHU,trangnguyen.n3t,Trang Nguyễn,Trang Nguyễn,NaN,False
4,Learner_Gr2,U01UJ9M5TU5,minhnguyentk95,Minh Nguyen,Nguyen Thi Kim Minh,NaN,False
5,Learner_Gr1,U01UMBXJLHG,nvlinh.dth,Nguyen Linh,Nguyen Linh,NaN,False
6,Learner_Gr4,U01UMC08KL2,auslyn.nguyen,NaN,Minh-Thu Nguyen Hoang,NaN,False
7,Learner_Gr1,U01UMC0N5U2,chuviethong.vn,Việt Hồng Chu,Việt Hồng Chu,Business Analyst,False
8,Learner_Gr1,U01USGKQ771,trang.ngo1709,Trang,Ngo Thi Minh Trang,NaN,False
9,Learner_Gr3,U01UTGRP0HJ,phamtran30996,Tran Pham,Tran Pham,NaN,False


In [323]:
### 5. Learner nào chưa nộp bất cứ bài tập nào?
print ('Learner chưa nộp bài nào:')
SQL_5 ='''
SELECT *
FROM members_df AS t1
WHERE t1.real_name NOT IN (
    SELECT 
    t2.real_name
    FROM asgmt_submitted AS t2
    )
AND DataCracy_role LIKE 'Learner_Gr%'
'''
con.execute(SQL_5).fetch_df()

Learner chưa nộp bài nào:


,DataCracy_role,user_id,name,display_name,real_name,title,is_bot
0,Learner_Gr1,U01U8CAMGB1,huyenngoc2017,Ngoc Huyen,Ngoc Huyen,NaN,False
1,Learner_Gr1,U01UMBXJLHG,nvlinh.dth,Nguyen Linh,Nguyen Linh,NaN,False
2,Learner_Gr1,U01VBFY2E59,trangh,Gia Hoang Tran,Gia Hoang Tran,NaN,False
3,Learner_Gr4,U0233KHLM2M,vuthanhdat.contact,NaN,Vuthanhdat Contact,NaN,False


In [301]:
## 6. Tỷ lệ % Learner đã nộp assignment 1, 2, 3?
print ('Tỉ lệ nộp bài theo từng assignment (count tất cả msg):')
SQL_6 = '''
WITH submit_percent AS (
  SELECT 
  t1.*,
  t2.channel_name
  FROM msg_df AS t1
  JOIN channel_df AS t2
  ON t1.channel_id = t2.channel_id
  WHERE t1.user_id  IN (
    SELECT 
    user_id
    FROM members_df
    WHERE DataCracy_role LIKE 'Learner_Gr%'
  ) AND t2.channel_name LIKE 'atom-assignment%'
),
total AS (
    SELECT 
    COUNT(real_name) as total_learner
    FROM members_df
    WHERE DataCracy_role LIKE 'Learner_Gr%'
)
SELECT 
  channel_name,
  COUNT(msg_id) / cast(total_learner as float) * 100 AS ratio_sub
FROM submit_percent, total
WHERE msg_id IS NOT NULL
GROUP BY channel_name, total_learner
  
'''
con.execute(SQL_6).fetch_df()

Tỉ lệ nộp bài theo từng assignment (count tất cả msg):


,channel_name,ratio_sub
0,atom-assignment1,97.435898
1,atom-assignment2,87.179489
2,atom-assignment3,69.230774
3,atom-assignment4,12.820514


In [427]:
### 7. Tỷ lệ % Learner đã submit bài và dc review trong assignment 1, 2, 3?
print ('Tỉ lệ nộp bài theo từng assignment đã được review (chỉ tính msg có link git):')
SQL_7 = '''
WITH submit_review AS (
    SELECT *
    FROM asgmt_submitted --------------------------> sử dụng lại bảng đã tạo ở trên gồm các learner đã nộp bài trong assignments
    WHERE reply_user_count > 0
),
total AS (
    SELECT 
    COUNT(real_name) as total_learner
    FROM members_df
    WHERE DataCracy_role LIKE 'Learner_Gr%'
)
SELECT 
channel_name,
COUNT(real_name) / cast(total_learner as float) * 100 AS ratio_sub
FROM submit_review, total
GROUP BY channel_name, total_learner
'''
con.execute(SQL_7).fetch_df()

Tỉ lệ nộp bài theo từng assignment đã được review (chỉ tính msg có link git):


,channel_name,ratio_sub
0,atom-assignment1,64.102570
1,atom-assignment2,76.923080
2,atom-assignment3,28.205130
3,atom-assignment4,2.564103


In [410]:
con.execute( '''
    CREATE TABLE gr_done as (
    SELECT
    channel_name,
    DataCracy_role,
    Count(real_name) as submitted
    FROM asgmt_submit 
    group by DataCracy_role, channel_name
    )
    '''
).fetch_df()

,Count
0,15


In [416]:
con.execute( '''
CREATE TABLE gr_mem as (
    SELECT
    DataCracy_role,
    COUNT(real_name) as total_learner
    FROM members_df
    WHERE DataCracy_role LIKE 'Learner_Gr%'
    group by DataCracy_role
) '''
).fetch_df()

,Count
0,4


In [430]:
### 8. Learners theo Group nào có tỷ lệ % hoàn thành bài tập cao nhất?
print ('Group có tỷ lệ % hoàn thành bài tập cao nhất:')
SQL_8 = '''

WITH gr_learner AS (
SELECT
  t1.*,
  t2.total_learner
FROM gr_done as t1
JOIN gr_mem as t2
ON t1.DataCracy_role = t2.DataCracy_role
)
SELECT *,
    submitted / cast(total_learner as float) * 100 AS ratio_submit
    FROM gr_learner
    ORDER BY ratio_submit DESC
'''
con.execute(SQL_8).fetch_df()

Group có tỷ lệ % hoàn thành bài tập cao nhất:


,channel_name,DataCracy_role,submitted,total_learner,ratio_submit
0,atom-assignment1,Learner_Gr3,9,9,100.000000
1,atom-assignment2,Learner_Gr2,9,9,100.000000
2,atom-assignment2,Learner_Gr3,8,9,88.888893
3,atom-assignment3,Learner_Gr2,8,9,88.888893
4,atom-assignment2,Learner_Gr4,8,10,80.000000
5,atom-assignment3,Learner_Gr3,7,9,77.777779
6,atom-assignment1,Learner_Gr4,7,10,70.000000
7,atom-assignment3,Learner_Gr4,7,10,70.000000
8,atom-assignment1,Learner_Gr2,6,9,66.666672
9,atom-assignment1,Learner_Gr1,7,11,63.636364
